# Desafio 3
# =========
* Deserializar un JSON

In [1]:
import os
import requests
import json
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

22/01/24 09:14:52 WARN Utils: Your hostname, fede-dsk resolves to a loopback address: 127.0.1.1; using 192.168.1.29 instead (on interface wlxf8d111634ba2)
22/01/24 09:14:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/24 09:14:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Read JSON file into dataframe
df = spark.read.json("./input_ejemplos/Sellers.json")
df.printSchema()
#df.show()

root
 |-- body: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |-- buyer_reputation: struct (nullable = true)
 |    |    |-- tags: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- country_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- logo: string (nullable = true)
 |    |-- nickname: string (nullable = true)
 |    |-- permalink: string (nullable = true)
 |    |-- points: long (nullable = true)
 |    |-- registration_date: string (nullable = true)
 |    |-- seller_reputation: struct (nullable = true)
 |    |    |-- level_id: string (nullable = true)
 |    |    |-- power_seller_status: string (nullable = true)
 |    |    |-- transactions: struct (nullable = true)
 |    |    |    |-- canceled: long (nullable = true)
 |    |    |    |-- completed: long (nullable = true)
 |    |    |    |-- period: s

In [62]:
df_select = df.select("body.site_id", "body.id", "body.nickname", 'body.points')
df_select = df_select \
            .withColumnRenamed("body.site_id","siteId") \
            .withColumnRenamed("body.id","sellerId") \
            .withColumnRenamed("nickname","sellerNickname") \
            .withColumnRenamed("points","sellerPoints")
df_select.show()

+-------+---------+--------------------+------------+
|site_id|       id|      sellerNickname|sellerPoints|
+-------+---------+--------------------+------------+
|    MPE|298734964|      MARIELATAQUIRE|           2|
|    MPE|183049329|         MURO8709951|          -3|
|    MPE| 94592189|          ILLARYPERU|          -2|
|    MPE|520133997|     ISABELLADELPOZO|           1|
|    MPE|684964436|         PHMO1747353|           0|
|    MPE|685079498|MELISSASUSANAARVA...|           0|
|    MPE|646068761|   YOMIDELGADOSNCHEZ|           0|
|    MPE|685310649|DONATILDONATILDEC...|           0|
|    MPE|685419864|        VANESSAURNER|           0|
|    MPE|285674870|    ERICKLOPEZUSMSYA|           2|
|    MPE|685275449|DANIELARUIZRIDRIGUES|           0|
|    MPE| 48893023|        MARCELASUSAN|           1|
|    MPE|603331827|         COVA1031117|           0|
|    MPE|205264135|         GOBR7283790|           1|
|    MPE|580279940|        LORDVENCEDOR|           0|
|    MPE|300834652| DANIELAH

## Saving CSV

In [66]:
positivo = df_select.filter(df_select.sellerPoints > 0)
cero = df_select.filter(df_select.sellerPoints == 0)
negativo = df_select.filter(df_select.sellerPoints < 0)

csvs = ['positivo', 'cero', 'negativo']
# Creo csv con 1 partición
for csv in csvs:
    eval(csv) \
    .repartition(1) \
    .write.format("com.databricks.spark.csv") \
    .option("header", "true") \
    .save(f'MPE/2022/01/24/{csv}/data.csv')

## Using Schema

In [20]:
# display dataframe columns
df.schema.fields

[StructField(body,StructType(List(StructField(address,StructType(List(StructField(city,StringType,true),StructField(state,StringType,true))),true),StructField(buyer_reputation,StructType(List(StructField(tags,ArrayType(StringType,true),true))),true),StructField(country_id,StringType,true),StructField(id,LongType,true),StructField(logo,StringType,true),StructField(nickname,StringType,true),StructField(permalink,StringType,true),StructField(points,LongType,true),StructField(registration_date,StringType,true),StructField(seller_reputation,StructType(List(StructField(level_id,StringType,true),StructField(power_seller_status,StringType,true),StructField(transactions,StructType(List(StructField(canceled,LongType,true),StructField(completed,LongType,true),StructField(period,StringType,true),StructField(ratings,StructType(List(StructField(negative,LongType,true),StructField(neutral,LongType,true),StructField(positive,LongType,true))),true),StructField(total,LongType,true))),true))),true),Struc

In [49]:
from pyspark.sql.types import *

structureSchema = StructType([
        StructField('body', StructType([
                StructField("site_id",StringType(),True),
                StructField("id",IntegerType(),True),
                StructField("nickname",StringType(),True),
                StructField("points",LongType(),True)
             ]))
         ])

In [51]:
df_with_schema = spark.read.schema(structureSchema) \
        .json("./input_ejemplos/Sellers.json")
df_with_schema.printSchema()

root
 |-- body: struct (nullable = true)
 |    |-- site_id: string (nullable = true)
 |    |-- id: integer (nullable = true)
 |    |-- nickname: string (nullable = true)
 |    |-- points: long (nullable = true)



In [53]:
df_with_schema.select("body.site_id", "body.id", "body.nickname", 'body.points').show()

+-------+---------+--------------------+------+
|site_id|       id|            nickname|points|
+-------+---------+--------------------+------+
|    MPE|298734964|      MARIELATAQUIRE|     2|
|    MPE|183049329|         MURO8709951|    -3|
|    MPE| 94592189|          ILLARYPERU|    -2|
|    MPE|520133997|     ISABELLADELPOZO|     1|
|    MPE|684964436|         PHMO1747353|     0|
|    MPE|685079498|MELISSASUSANAARVA...|     0|
|    MPE|646068761|   YOMIDELGADOSNCHEZ|     0|
|    MPE|685310649|DONATILDONATILDEC...|     0|
|    MPE|685419864|        VANESSAURNER|     0|
|    MPE|285674870|    ERICKLOPEZUSMSYA|     2|
|    MPE|685275449|DANIELARUIZRIDRIGUES|     0|
|    MPE| 48893023|        MARCELASUSAN|     1|
|    MPE|603331827|         COVA1031117|     0|
|    MPE|205264135|         GOBR7283790|     1|
|    MPE|580279940|        LORDVENCEDOR|     0|
|    MPE|300834652| DANIELAHILARIORAMOS|     0|
|    MPE|270322958|JHONANTHONYCAYLLA...|     0|
|    MPE|684554092|OSORIOCOLQUIJESSM...|